In [ ]:
# IDGB source
# Sur la base de vgsales_at
# récupération de title|platform|franchise|rating|nb_rating|nb_follower

In [ ]:
import requests
import json
from datetime import datetime
from datetime import date
from time import sleep
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from os import listdir
import json


In [ ]:
# https://api-docs.igdb.com/
# https://www.igdb.com/discover
# https://github.com/twitchtv/igdb-api-python
# https://dev.twitch.tv/console

In [ ]:
with open('credentials_igdb.json', 'r') as f:
    credentials = json.load(f)
    
client_id = credentials['client_id']
client_secret = credentials['client_secret']
auth_token = credentials['auth_token']

In [ ]:
# filtre sur les plateformes de metacritic
['Nintendo 64', 'PlayStation', 'PlayStation 3', 'Dreamcast',
 'Xbox 360', 'Wii', 'Xbox One', 'PC', 'Switch', 'PlayStation 2',
 'PlayStation 4', 'GameCube', 'Xbox', 'Wii U', 'Game Boy Advance',
 '3DS', 'Xbox Series X', 'DS', 'PlayStation Vita', 'PlayStation 5',
 'PSP', 'Stadia']

In [ ]:
# envoie de la requete
def igdb_request(req):
    url        = "https://api.igdb.com/v4/games"       
    
    request_params = { 'headers': { 'Client-ID': client_id,
                                    'Authorization': 'Bearer ' + auth_token },
                       'data':  req}
    
    r = requests.post(url, **request_params)
    r_dictionary = json.loads(r.text)
    
    return len(r_dictionary) , r_dictionary

# Construction de la requete
def get_game_request(name):
    t  = f'search \"{name}\"; '
    t += 'fields slug, name, rating, rating_count, follows, platforms.name, franchises.name, release_dates.date, release_dates.platform; '
    t += 'limit 5;'
    return t

# fields follows, url, id, name, hypes, multiplayer_modes.*, involved_companies.company.name, \
# rating_count, rating, platforms.name, game_engines.name, genres.name, release_dates.date, franchises.name; \
# alternative_names.*, multiplayer_modes.*,


In [ ]:
# Manual request
titre='Grand Theft Auto: San Andreas'
counter, game_dict = igdb_request(get_game_request(titre))
game_dict

In [ ]:
# methode d'enregistrement des données
def save_game(result):
    if len(result):
        file_name = result[0]['slug']
        with open('igdb/' + file_name + '.json', 'w') as outfile:
            json.dump(result, outfile)

In [ ]:
vgchartz = pd.read_csv("PyVG-Data-Science-to-predict-Video-Games-sales/data/vgchartz.csv", sep=",")
print (vgchartz.shape)
vgchartz.head()

In [ ]:
list_jeux_df = vgchartz.Name.unique()
print (len(list_jeux_df))
list_jeux_df[:10]

In [ ]:
vgchartz["title_norm"] = vgchartz["Name"].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
vgchartz["title_norm"] = vgchartz["title_norm"].str.lower()
vgchartz.head()

In [ ]:
list_jeux_df = vgchartz.title_norm.unique()
print (len(list_jeux_df))
list_jeux_df[:10]

In [ ]:
for titre in tqdm(list_jeux_df[1885:1886]):
    titre = titre.replace("\"","")
    counter, game_dict = igdb_request(get_game_request(titre))
    if counter > 0: save_game(game_dict)
    sleep(0.5)

### Partie lecture des fichiers scrapés

In [ ]:
d_platf = {3:'Linux', 4:'N64', 5:'Wii', 6: 'PC', 7:'PS', 8:'PS2', 9:'PS3', 11: 'XB', 12: 'X360', 18:'SNES', 20:'DS',
      21:'GC', 22:'GBC', 24:'GBA', 33:'GB', 39:'iOS', 170:'Stadia',
      37:'3DS',
     38: 'PSP', 41:'WiiU', 46:'PSV', 48:'PS4',
      49: 'XOne', 130:'NS' ,167:'PS5', 169:'XS', 159:"DSi" }

# liste les noms des plateformes utilisées en lieu et place de celle de IGDB

In [ ]:
game_key = []
igdb_name = []
igdb_franchise  = []
igdb_follows  = []
igdb_platform  = []
igdb_release = []

d_platforms = {}

for fic in tqdm(listdir('igdb')):
    
    with open('igdb/' + fic) as json_file:
  
        data_json = json.load(json_file)
        #print (data_json, len(data_json))
           
        for g in data_json:
            
            p_game_key = g['slug']
            p_name = g['name']

            p_platforms = g['platforms'] if 'platforms' in g else []
            for elt in p_platforms:
                d_platforms[elt['id']]= elt['name']
        
            p_releases = g['release_dates'] if 'release_dates' in g else []
            d_releases = {}
            for elt in p_releases:
                try:
                    d_releases[elt['platform']]= elt['date']
                except:
                    pass
            
            p_franchise = g['franchises'][0]['name'] if 'franchises' in g else np.nan
            p_follows = g['follows'] if 'follows' in g else 0
            
            #print (p_game_key,p_name,p_follows,sep=' -> ')
            #print (p_franchise)
            #print (d_platforms)
            #print (d_releases)
            #print (p_follows)
            
            for pp in d_platforms.keys():
                # get rel date
                rd = -1
                try:
                    rd = d_releases[pp]
                except:
                    pass
            
                if rd != -1 and (pp in d_platf) and (p_game_key not in game_key):
                    
                    game_key.append(p_game_key)
                    igdb_name.append(p_name)
                    igdb_follows.append(p_follows)
                    igdb_franchise.append(p_franchise)
                    igdb_platform.append(d_platf[pp])
                    igdb_release.append(rd)
    
#print (d_platforms)

igdb = pd.DataFrame({'game_key' : game_key,
                     'Name': igdb_name,
                     'Platform': igdb_platform,
                     'Franchise': igdb_franchise,
                     'Follows' : igdb_follows,
                     'Release date': igdb_release
             })
print ()
print (igdb.shape)
igdb.head()
# (28930, 6)

In [ ]:
igdb.info()

### Corrections manuelles au fur et à mesure des trouvailles ;) 

In [ ]:
igdb.loc[igdb['Franchise']=="Rosario + Vampir",'Franchise'] = "Rosario + Vampire"
igdb.loc[igdb['Name'].str.contains('007 Legends'),'Franchise'] = "James Bond"
igdb.loc[igdb['game_key']=='tetris-attack--1','Release date'] = 329436288 # was 32943628800
igdb.loc[igdb['game_key']=='7-wonders-of-the-ancient-world-93a9164c-906f-4727-948a-14dade2fbfa2','game_key'] = '7-wonders-of-the-ancient-world'

igdb.loc[igdb['game_key']=='a-witchs-tale','game_key'] = 'a-witch-s-tale'
igdb.loc[igdb['game_key']=='3d-classics-kirbys-adventure','game_key'] = '3d-classics-kirby-s-adventure'


### Calcul du nombre de franchises

In [ ]:
igdb['N_Franchises'] = 1
ff = igdb.groupby('Franchise_wikipedia',as_index=False).agg({'N_Franchises':sum}).sort_values('N_Franchises')
print (ff.shape)
#(15997, 2)

In [ ]:
# ajout au dataframe
 # Platform pas utile car gérées avec des -- dans le noms des clé !
igdb.drop(columns=['Franchise','N_Franchises','Platform'],inplace=True)
igdb = igdb.merge(ff,on='Franchise_wikipedia',how='left')
igdb.shape
# (28930, 7)

### Mise en forme de la date

In [ ]:
igdb['Release_date'] = pd.to_datetime(igdb['Release date'],unit='s')
igdb.drop(columns=['Release date'],inplace=True)


### Suppression des --x qui sont ajoutés pour chaque plateforme

In [ ]:
igdb['game_key'].replace(r'--\d+$','',regex=True, inplace=True)
igdb[igdb['game_key'].str.contains('--')]
igdb.drop_duplicates(subset='game_key',inplace=True)
igdb['game_key'].duplicated().value_counts()

In [ ]:
igdb.info()
# 28176 

In [ ]:
igdb.head()

In [ ]:
platformes = ['2600','3DO','3DS','5200','7800','ACPC','AJ','AST','Aco','All','Amig','And','ApII','Arc','BBCM',
             'BRW','C128','C64','CD32','CDi','CV','DC','DS','DSi','DSiW','FMT','GB','GBA','GBC','GC','GEN','GG',
             'GIZ','Int','Linux','Lynx','MS','MSD','MSX','Mob','N64','NES','NG','NGage','NS','OR','OSX','Ouya',
             'PC','PCE','PCFX','PS','PS2','PS3','PS4','PS5','PSN','PSP','PSV','S32X','SAT','SCD','SNES','Series',
             'TG16','VB','VC','WS','WW','Wii','WiiU','WinP','X360','XB','XBL','XOne','XS','ZXS','iOS','iQue']

def check_platform(l):
    counter = 0
    print ('Verification des plateformes')
    for p in l:
        if p not in platformes:
            print (f' > {p} pas dans la liste des plateformes !')
            counter += 1
            
    print (f'Il y a {counter} plateforme(s) non conforme(s)')

check_platform([d_platf[i] for i in d_platf.keys()])

In [ ]:
igdb.to_csv("PyVG-Data-Science-to-predict-Video-Games-sales/data/igdb_franchises.csv", sep=",", encoding='utf-8', index=False)

In [ ]:
igdb[igdb['game_key'].str.contains(r'\w{2}s-')]